In [6]:
import numpy as np
import itertools
from collections import Counter
import sys
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
class MMboard:
    '''An instance of a MasterMind board.
    
    The object of the game is to guess the hidden code of length L, 
    where each code element is any one of C characters. 
    
    In response to a guess of the code, if the code is incorrect,
    a response (b, w) is provided; b represents the number of 
    characters where the correct character is provided in the 
    correct position, whereas w represents the number of characters
    that are in the code but are in the wrong position (i.e. the
    number of characters that are actually in the code but are not
    in the position guessed). 
    
    The game continues until the correct code is guessed, or until
    a predefined number of allowable tries have been exhausted. 
    
    Note: currently supports numcolors<=10. '''
    
    def __init__ (self, codelength=4, numcolors=6, max_tries=10, suppress_output=False):
        
        assert (codelength >= 1) and (max_tries >= 1)
        assert (numcolors >= 1)  and (numcolors <= 10)
        
        self._L = codelength
        self._C = numcolors
        self._N_iters = max_tries
        self._nooutput = suppress_output
        
        self._code = np.zeros(self._L)  # contains the code
        
        self.n_guessed = 0  # number of guesses tried
        self.gameover = False
    
    def _codeOK(self, cc):
        '''Helper function to check that inputs for code are in
        proper form. '''
        
        try:
            init = np.array([int(item) for item in cc])
        except:
            raise ValueError('Code not in the form of a list/array of length ' + str(self._L))

        if len(init) != self._L:
            raise ValueError('Code not of length ' + str(self._L))

        if not all(isinstance(item, int) for item in init):
            raise ValueError('Each character must be an integer, between 0 and ' + str(self._C - 1))

        if not((init >= 0).all() and (init <= self._C - 1).all()):
            raise ValueError('Each integer must be between 0 and ' + str(self._C - 1)) 
        
        return True
        
    def set_code(self, custom=None, showcode=True):
        '''Initalize/reset the code.
        
        Can set the code manually (provide a list or array of length L)
        or set one at random (default).
        
        Example: myboard.set_code([0, 1, 2, 3])
        
        showcode: whether or not the code being set is displayed
        '''
        
        if (custom is not None) and self._codeOK(custom):  
            self._code = np.array([int(item) for item in custom])
        else:
            self._code = np.random.randint(0, self._C, size=self._L)  # each element is [0, C)
        
        if showcode and (not self._nooutput):
            print "Code successfully initialized to ", self._code, "\n"
        elif not self._nooutput:
            print "Code successfully initialized. Good luck.\n"
            
        self.n_guessed = 0  # reset guess counter
        self.gameover = False
        
    def check_guess(self, guess, answer):
        '''Process a guess given the correct code.
        
        Take a guess in the form of a list of integers, and returns
        (number of characters in the correct position,
         number of characters in the wrong position but elsewhere in code).
         
        Usually, the answer will be the secret code (self._code), but this
        method can be used to compare any guess to any answer.
        
        Example: myboard.guess_code([0, 1, 2, 3], self._code)
        Returns: (1, 1)  # self._code is [2 1 4 4] '''
        
        # counters
        corpos = 0  
        wrongpos = 0  
        # track non-counted characters
        code_left = []
        guess_left = []
       
        # check for correct digit in correct place  
        for i, digit in enumerate(answer):            
            if digit==guess[i]:
                corpos += 1  
            else:
                code_left.append(digit)
                guess_left.append(guess[i])
            
        assert len(code_left) == len(guess_left)

        if len(code_left)>0:
            # check for correct digit in wrong place
            for digit in code_left:
                if digit in guess_left:
                    wrongpos += 1
                    guess_left.remove(digit) # removes only one occurrence
        
        return (corpos, wrongpos)

    def guess_code(self, guess):
        '''Entry method to process guess, with a check to see if number of guesses has
        been exceeded, or code guessed correctly.
        
        If game is over, self.gameover is set to True.
        
        Returns: a tuple - (number of characters in the correct position,
                            number of characters in the wrong position but elsewhere in code); 
                  None if there was an error in the guess; or
                  -1 if game is over'''

        if not self.gameover:
            try:
                assert self._codeOK(guess)
                self.n_guessed += 1
                if not self._nooutput:
                    print "guess #" + str(self.n_guessed) + " of " + str(self._N_iters) + ": you guessed ", guess 
                    sys.stdout.flush()
            except:
                if not self._nooutput:
                    print "  [Error] please enter a list of %i integers from 0 to %i. try again" % (self._L, self._C - 1)
                return None
        
            # get response
            b, w = self.check_guess(guess=guess, answer=self._code)
            assert (b + w) <= self._L

            if b == self._L:
                if not self._nooutput:
                    print "You have %i right item(s) in the right place" % b
                    print "You win!"
                self.gameover = True
                return (b, w)
            
            if not self._nooutput:                
                print "You have %i right item(s) in the right place, and" % b
                print "  %i right item(s) but in the wrong place\n" % w 
            
            if self.n_guessed == self._N_iters:
                if not self._nooutput:
                    print "Game over. The correct code was", self._code  
                self.gameover = True

            return (b, w)
        
        # else
        return -1  # game already over

## General notation

<p>Let us introduce some notation. For simplicity, we use one-indexed arrays even though the Python code is zero-indexed. We define:</p>
<ul>
<li>the list of possible colors as $CC = \{1, ..., C\}$</li>
<li>the list of positions in the code as $LL = \{1, ..., L\}$</li>
<li>the hidden code as $H_i \ \forall i \in LL$</li>
<li>a particular guess of the hidden code as $T_i \ \forall i \in LL$</li>
<li>the indicator function $\mathbb{1}_{A=B}$, which equals 1 if A=B and equals 0 otherwise</li>
</ul>
<p>Using the above notation, we can denote the responses at each turn as follows:</p>
<p>The number of correctly guessed pegs can be denoted as $B = \sum_{i=1}^L \mathbb{1}_{T_i=H_i} \ \forall i \in LL$.</p>
<p>The number of correctly guessed pegs in the wrong position can be denoted as: $W = \sum_{i=1}^{C} \min(\sum_{j=1}^{L}\mathbb{1}_{H_j=i, G_i}, \sum_{j=1}^{L}\mathbb{1}_{T_j=i, G_i}) - B$</p>

## Random sampling from posterior

<p><em>Adapted from http://staff.utia.cas.cz/vomlel/mastermind.pdf</em></p>

<p>Using the above notation, we can define the joint probability distribution over all possible code sequences as $P(H_1, ..., H_L)$.</p>
<p>Our prior is uniformly distributed, i.e. all $P(H_1=h_1, ..., H_L=h_l) = \frac{1}{C^L}$</p>
<p>We denote the evidence that we obtain at each step as $e = (B, W)$, where B and W are defined as above. We can update the posterior joint distribution over code sequences as follows:</p>
$P(H_1=h_1, ..., H_L=h_l | e) = \left\{
\begin{align*}
\frac{1}{\big| \ s(e) \ \big|} & & if (h_1, ..., h_l) \ \text{is a possible code}\\
0 & & \text{otherwise}
\end{align*}
\right.$
<p>where s(e) denotes the set of possible hidden codes, given the evidence, and | s(e) | denotes the cardinality of this set.</p>
<p>We can define the posterior after multiple game steps analogously:</p>
$P(H_1=h_1, ..., H_L=h_l | e_1, ..., e_n) = \left\{
\begin{align*}
\frac{1}{\big| \ s(e_1) \ \cap \ ... \ \cap \ s(e_n) \ \big|} & & if (h_1, ..., h_l) \ \text{is a possible code}\\
0 & & \text{otherwise}
\end{align*}
\right.$
<p>where $\ s(e_1) \ \cap \ ... \ \cap \ s(e_n)$ denotes the intersection of the sets of possible hidden codes, given the evidence at each step, and the entire denominator denotes the cardinality of this intersection.</p>
<p>We can use this setup to define the posterior updates at each round of the game. We note that the posterior distribution remains uniform among valid codes at each step of the game, so the best we can do at each round is to choose randomly based on this updated distribution.</p>
<p>An aside: some of the literature on Mastermind uses Shannon entropy to measure the uncertainty associated with a given code. This measure is calculated as follows for a given code sequence: $P(H_1=h_1, ..., H_L=h_l | e_1, ..., e_n) \log P(H_1=h_1, ..., H_L=h_l | e_1, ..., e_n)$ (or, in some texts, the negative of this value) and is only equal to zero where the code is known. However, we do not use this measure here as it cannot be used to differentiate across potential guesses due to the uniform posterior updates under this framework.</p>

## Local entropy

Alternative: guess based on entropy of possible response classes  
http://www.geometer.org/mathcircles/mastermind.pdf